In [ ]:
import os
import numpy as np
import pandas as pd


In [ ]:
# WRITE PATH TO DATA DIRECTORY BELOW 

data_path = '../data/redclassic/'

files = []
data = {}
files_received = []
load_files = []
driver_files = []

try:
    for i in os.listdir(data_path):
        if os.path.isfile(os.path.join(data_path,i)):
            files.append(i)
except FileNotFoundError:
    print("PLEASE MAKE SURE THE DATA PATH LEADS TO THE DATA DIRECTORY WHERE THE DATA FILES ARE LOCATED\n")   
except NameError:
    print("PLEASE INSERT A VALID PATH STRING TO THE DATA DIRECTORY\n")

if '.DS_Store' in files:
    files.remove('.DS_Store')
for file in files:
    data[file] = pd.read_csv(data_path + file, low_memory=False)

for file in files:
    try:
        data[file]
        files_received.append(file)
        if 'driver' in file.lower():
            driver_files.append(file)
        else:
            load_files.append(file)
    except KeyError:
        print(f'** The {file} file is missing from the data folder **'.upper())        

if len(driver_files) == 1:
    print(f"Received {len(driver_files)} driver file")
else:
    print(f"Received {len(driver_files)} driver files")
    
if len(load_files) == 1:
    print(f"Received {len(load_files)} load file")
else:
    print(f"Received {len(load_files)} load files")

if len(files_received) == 0:
    print("\n!!! NO FILES RECEIVED !!!")
    
for file in files_received:
    print(f"{file} - {type(data[file])}")

In [ ]:
# Combine all files of similar type to one file
load_data_combined_list = [data[file] for file in load_files]
    
load_data_combined_df = pd.concat(load_data_combined_list, ignore_index=True)

print(load_data_combined_df.info())
print(load_data_combined_df.head())

In [ ]:
# Run Load Files Checks

# LIST THE COLUMNS REQUIRED (NO BLANKS)
required_load_columns = []

for file in load_files:
    df = data[file]
    print(df.info())

In [ ]:
# Run Driver Files Checks
blanks = {}

required_driver_columns = list(data[driver_files[0]].columns.values)

# Columns to remove from required list
columns_not_required = required_driver_columns[7:]

for file in driver_files:
    df = data[file]
    print(df.info())
    
#     Find Blanks for required columns
    for column in required_driver_columns:
        if column in columns_not_required:
            pass
        else:
            column_blanks_df = df[df[column].isnull()]
            blanks[column] = column_blanks_df

for column in blanks.keys():
    if blanks[column].empty == False:
        print(blanks[column])
    else:
        pass